## Introduction

Image detection is a good entry point for learning about the use of neural networks, which I aim to learn more about in this project. The problem to be solved in my project is image detection (of handwritten digits, in this case) by using the branch of machine learning that deals with image detection. 

## Data Wrangling

I'll first read in the data and then check for any anomalies. Reading in the data entails a series of functions, the first of which loads in raw gzip file and serializes it from a hierarchy to a series of bytes. That adjustment will make the data easier to vectorize and put into arrays afterward.

In [10]:
import numpy as np
import gzip
import pickle #converts object hierarchy into byte stream

def load_mnist():
    file = gzip.open('mnist.pkl.gz', 'rb') #read in in binary mode
    train, test = pickle.load(file)
    file.close()
    return (train, test)

The next function creates training and test datasets that will be easier to process. The initial training data is a list of 50,000 (x, y) tuples, where x is 784-dimensional (from 28x28 pixels) array of input image and y is a ten-dimensional array capturing the corresponding classification (digit value). The initial test data has the same structure but is a list of 10,000 tuples. 

In [11]:
def data_wrapper():
    train, test = load_mnist()
     #reshape first value (x) in each training tuple into 784D (28x28 pixels) vector
    train_inputs = [np.reshape(x, (784, 1)) for x in train[0]] 
    train_results = [vectorized_result(y) for y in train[1]]
    train_data = zip(train_inputs, train_results)
#validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
#validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in test[0]]
    test_data = zip(test_inputs, test[1])
    return (train_data, test_data) #validation_data,

The third formula converts a 0-9 digit into a corresponding desired output from the neural network using 1 as a binary marker among zeroes otherwise.

In [12]:
def vectorized_result(j):
    tenD_uv = np.zeros((10, 1)) #create 10D unit vector
    tenD_uv[j] = 1.0 #put 1 in jth position
    return tenD_uv